In [13]:
'''
A Dynamic Recurrent Neural Network (LSTM) implementation example using
TensorFlow library. This example is using a toy dataset to classify linear
sequences. The generated sequences have variable length.
Long Short Term Memory paper: http://deeplearning.cs.cmu.edu/pdfs/Hochreiter97_lstm.pdf
Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''

from __future__ import print_function

import tensorflow as tf
import random


# ====================
#  TOY DATA GENERATOR
# ====================
class ToySequenceData(object):
    """ Generate sequence of data with dynamic length.
    This class generate samples for training:
    - Class 0: linear sequences (i.e. [0, 1, 2, 3,...])
    - Class 1: random sequences (i.e. [1, 3, 10, 7,...])
    NOTICE:
    We have to pad each sequence to reach 'max_seq_len' for TensorFlow
    consistency (we cannot feed a numpy array with inconsistent
    dimensions). The dynamic calculation will then be perform thanks to
    'seqlen' attribute that records every actual sequence length.
    """
    def __init__(self, n_samples=1000, max_seq_len=20, min_seq_len=3,
                 max_value=1000):
        self.data = []
        self.labels = []
        self.seqlen = []
        for i in range(n_samples):
            # Random sequence length
            len = random.randint(min_seq_len, max_seq_len)
            # Monitor sequence length for TensorFlow dynamic calculation
            self.seqlen.append(len)
            # Add a random or linear int sequence (50% prob)
            if random.random() < .5:
                # Generate a linear sequence
                rand_start = random.randint(0, max_value - len)
                s = [[float(i)/max_value] for i in
                     range(rand_start, rand_start + len)]
                # Pad sequence for dimension consistency
                s += [[0.] for i in range(max_seq_len - len)]
                self.data.append(s)
                self.labels.append([1., 0.])
            else:
                # Generate a random sequence
                s = [[float(random.randint(0, max_value))/max_value]
                     for i in range(len)]
                # Pad sequence for dimension consistency
                s += [[0.] for i in range(max_seq_len - len)]
                self.data.append(s)
                self.labels.append([0., 1.])
        self.batch_id = 0

    def next(self, batch_size):
        """ Return a batch of data. When dataset end is reached, start over.
        """
        if self.batch_id == len(self.data):
            self.batch_id = 0
        batch_data = (self.data[self.batch_id:min(self.batch_id +
                                                  batch_size, len(self.data))])
        batch_labels = (self.labels[self.batch_id:min(self.batch_id +
                                                  batch_size, len(self.data))])
        batch_seqlen = (self.seqlen[self.batch_id:min(self.batch_id +
                                                  batch_size, len(self.data))])
        self.batch_id = min(self.batch_id + batch_size, len(self.data))
        return batch_data, batch_labels, batch_seqlen


# ==========
#   MODEL
# ==========

# Parameters
learning_rate = 0.01
training_iters = 1000000
batch_size = 128
display_step = 10

# Network Parameters
seq_max_len = 20 # Sequence max length
n_hidden = 64 # hidden layer num of features
n_classes = 2 # linear sequence or not

trainset = ToySequenceData(n_samples=1000, max_seq_len=seq_max_len)
testset = ToySequenceData(n_samples=500, max_seq_len=seq_max_len)

# tf Graph input
x = tf.placeholder("float", [None, seq_max_len, 1])
y = tf.placeholder("float", [None, n_classes])
# A placeholder for indicating each sequence length
seqlen = tf.placeholder(tf.int32, [None])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}


def dynamicRNN(x, seqlen, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, seq_max_len, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden)

    # Get lstm cell output, providing 'sequence_length' will perform dynamic
    # calculation.
    outputs, states = tf.contrib.rnn.static_rnn(lstm_cell, x, dtype=tf.float32,
                                sequence_length=seqlen)

    # When performing dynamic calculation, we must retrieve the last
    # dynamically computed output, i.e., if a sequence length is 10, we need
    # to retrieve the 10th output.
    # However TensorFlow doesn't support advanced indexing yet, so we build
    # a custom op that for each sample in batch size, get its length and
    # get the corresponding relevant output.

    # 'outputs' is a list of output at every timestep, we pack them in a Tensor
    # and change back dimension to [batch_size, n_step, n_input]
    outputs = tf.stack(outputs)
    outputs = tf.transpose(outputs, [1, 0, 2])

    # Hack to build the indexing and retrieve the right output.
    batch_size = tf.shape(outputs)[0]
    # Start indices for each sample
    index = tf.range(0, batch_size) * seq_max_len + (seqlen - 1)
    # Indexing
    outputs = tf.gather(tf.reshape(outputs, [-1, n_hidden]), index)

    # Linear activation, using outputs computed above
    return tf.matmul(outputs, weights['out']) + biases['out']

pred = dynamicRNN(x, seqlen, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y, batch_seqlen = trainset.next(batch_size)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       seqlen: batch_seqlen})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y,
                                                seqlen: batch_seqlen})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y,
                                             seqlen: batch_seqlen})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy
    test_data = testset.data
    test_label = testset.labels
    test_seqlen = testset.seqlen
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label,
seqlen: test_seqlen}))

c:\python35\lib\site-packages\tensorflow\python\ops\gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Iter 1280, Minibatch Loss= 0.707406, Training Accuracy= 0.49219
Iter 2560, Minibatch Loss= 0.687201, Training Accuracy= 0.48438
Iter 3840, Minibatch Loss= 0.681996, Training Accuracy= 0.49219
Iter 5120, Minibatch Loss= 0.694412, Training Accuracy= 0.56731
Iter 6400, Minibatch Loss= 0.697460, Training Accuracy= 0.54688
Iter 7680, Minibatch Loss= 0.682088, Training Accuracy= 0.53906
Iter 8960, Minibatch Loss= 0.685347, Training Accuracy= 0.50781
Iter 10240, Minibatch Loss= 0.691793, Training Accuracy= 0.55769
Iter 11520, Minibatch Loss= 0.694166, Training Accuracy= 0.50000
Iter 12800, Minibatch Loss= 0.680307, Training Accuracy= 0.59375
Iter 14080, Minibatch Loss= 0.686946, Training Accuracy= 0.54688
Iter 15360, Minibatch Loss= 0.690448, Training Accuracy= 0.58654
Iter 16640, Minibatch Loss= 0.692622, Training Accuracy= 0.51562
Iter 17920, Minibatch Loss= 0.679287, Training Accuracy= 0.59375
Iter 19200, Minibatch Loss= 0.687464, Training Accuracy= 0.54688
Iter 20480, Minibatch Loss= 0.68

Iter 162560, Minibatch Loss= 0.585067, Training Accuracy= 0.71875
Iter 163840, Minibatch Loss= 0.512428, Training Accuracy= 0.75962
Iter 165120, Minibatch Loss= 0.529889, Training Accuracy= 0.75781
Iter 166400, Minibatch Loss= 0.558748, Training Accuracy= 0.65625
Iter 167680, Minibatch Loss= 0.573395, Training Accuracy= 0.72656
Iter 168960, Minibatch Loss= 0.500318, Training Accuracy= 0.75000
Iter 170240, Minibatch Loss= 0.517690, Training Accuracy= 0.77344
Iter 171520, Minibatch Loss= 0.551897, Training Accuracy= 0.64062
Iter 172800, Minibatch Loss= 0.562360, Training Accuracy= 0.71094
Iter 174080, Minibatch Loss= 0.490633, Training Accuracy= 0.75000
Iter 175360, Minibatch Loss= 0.507366, Training Accuracy= 0.78125
Iter 176640, Minibatch Loss= 0.545925, Training Accuracy= 0.64062
Iter 177920, Minibatch Loss= 0.552397, Training Accuracy= 0.70312
Iter 179200, Minibatch Loss= 0.483211, Training Accuracy= 0.75000
Iter 180480, Minibatch Loss= 0.499021, Training Accuracy= 0.78125
Iter 18176

Iter 322560, Minibatch Loss= 0.427719, Training Accuracy= 0.83654
Iter 323840, Minibatch Loss= 0.440938, Training Accuracy= 0.82031
Iter 325120, Minibatch Loss= 0.501711, Training Accuracy= 0.71875
Iter 326400, Minibatch Loss= 0.484179, Training Accuracy= 0.75781
Iter 327680, Minibatch Loss= 0.426374, Training Accuracy= 0.83654
Iter 328960, Minibatch Loss= 0.439696, Training Accuracy= 0.82031
Iter 330240, Minibatch Loss= 0.500885, Training Accuracy= 0.71875
Iter 331520, Minibatch Loss= 0.483071, Training Accuracy= 0.75781
Iter 332800, Minibatch Loss= 0.425026, Training Accuracy= 0.83654
Iter 334080, Minibatch Loss= 0.438463, Training Accuracy= 0.81250
Iter 335360, Minibatch Loss= 0.500056, Training Accuracy= 0.72656
Iter 336640, Minibatch Loss= 0.481964, Training Accuracy= 0.76562
Iter 337920, Minibatch Loss= 0.423671, Training Accuracy= 0.83654
Iter 339200, Minibatch Loss= 0.437237, Training Accuracy= 0.81250
Iter 340480, Minibatch Loss= 0.499222, Training Accuracy= 0.72656
Iter 34176

Iter 482560, Minibatch Loss= 0.393474, Training Accuracy= 0.82812
Iter 483840, Minibatch Loss= 0.460165, Training Accuracy= 0.74219
Iter 485120, Minibatch Loss= 0.432039, Training Accuracy= 0.78125
Iter 486400, Minibatch Loss= 0.369329, Training Accuracy= 0.83654
Iter 487680, Minibatch Loss= 0.390632, Training Accuracy= 0.82812
Iter 488960, Minibatch Loss= 0.457157, Training Accuracy= 0.75781
Iter 490240, Minibatch Loss= 0.428290, Training Accuracy= 0.78125
Iter 491520, Minibatch Loss= 0.366020, Training Accuracy= 0.83654
Iter 492800, Minibatch Loss= 0.387551, Training Accuracy= 0.84375
Iter 494080, Minibatch Loss= 0.453872, Training Accuracy= 0.75000
Iter 495360, Minibatch Loss= 0.424206, Training Accuracy= 0.78125
Iter 496640, Minibatch Loss= 0.362453, Training Accuracy= 0.83654
Iter 497920, Minibatch Loss= 0.384199, Training Accuracy= 0.84375
Iter 499200, Minibatch Loss= 0.450271, Training Accuracy= 0.75781
Iter 500480, Minibatch Loss= 0.419783, Training Accuracy= 0.78906
Iter 50176

Iter 642560, Minibatch Loss= 0.203376, Training Accuracy= 0.95312
Iter 643840, Minibatch Loss= 0.136131, Training Accuracy= 0.96094
Iter 645120, Minibatch Loss= 0.146199, Training Accuracy= 0.99038
Iter 646400, Minibatch Loss= 0.175118, Training Accuracy= 0.97656
Iter 647680, Minibatch Loss= 0.193214, Training Accuracy= 0.96094
Iter 648960, Minibatch Loss= 0.125702, Training Accuracy= 0.96094
Iter 650240, Minibatch Loss= 0.139424, Training Accuracy= 0.99038
Iter 651520, Minibatch Loss= 0.169696, Training Accuracy= 0.97656
Iter 652800, Minibatch Loss= 0.185597, Training Accuracy= 0.96094
Iter 654080, Minibatch Loss= 0.118102, Training Accuracy= 0.96094
Iter 655360, Minibatch Loss= 0.134150, Training Accuracy= 0.99038
Iter 656640, Minibatch Loss= 0.164783, Training Accuracy= 0.97656
Iter 657920, Minibatch Loss= 0.179395, Training Accuracy= 0.96094
Iter 659200, Minibatch Loss= 0.112099, Training Accuracy= 0.96094
Iter 660480, Minibatch Loss= 0.129541, Training Accuracy= 0.99038
Iter 66176

Iter 802560, Minibatch Loss= 0.071222, Training Accuracy= 0.96094
Iter 803840, Minibatch Loss= 0.084168, Training Accuracy= 0.98077
Iter 805120, Minibatch Loss= 0.106694, Training Accuracy= 0.97656
Iter 806400, Minibatch Loss= 0.121377, Training Accuracy= 0.95312
Iter 807680, Minibatch Loss= 0.070632, Training Accuracy= 0.96094
Iter 808960, Minibatch Loss= 0.083385, Training Accuracy= 0.98077
Iter 810240, Minibatch Loss= 0.105516, Training Accuracy= 0.97656
Iter 811520, Minibatch Loss= 0.120407, Training Accuracy= 0.95312
Iter 812800, Minibatch Loss= 0.070067, Training Accuracy= 0.96094
Iter 814080, Minibatch Loss= 0.082635, Training Accuracy= 0.98077
Iter 815360, Minibatch Loss= 0.104562, Training Accuracy= 0.97656
Iter 816640, Minibatch Loss= 0.119471, Training Accuracy= 0.95312
Iter 817920, Minibatch Loss= 0.069526, Training Accuracy= 0.96094
Iter 819200, Minibatch Loss= 0.081915, Training Accuracy= 0.98077
Iter 820480, Minibatch Loss= 0.103913, Training Accuracy= 0.97656
Iter 82176

Iter 962560, Minibatch Loss= 0.068573, Training Accuracy= 0.99038
Iter 963840, Minibatch Loss= 0.078269, Training Accuracy= 0.98438
Iter 965120, Minibatch Loss= 0.100779, Training Accuracy= 0.95312
Iter 966400, Minibatch Loss= 0.059909, Training Accuracy= 0.96094
Iter 967680, Minibatch Loss= 0.068236, Training Accuracy= 0.99038
Iter 968960, Minibatch Loss= 0.077044, Training Accuracy= 0.98438
Iter 970240, Minibatch Loss= 0.100315, Training Accuracy= 0.95312
Iter 971520, Minibatch Loss= 0.059708, Training Accuracy= 0.96094
Iter 972800, Minibatch Loss= 0.067904, Training Accuracy= 0.99038
Iter 974080, Minibatch Loss= 0.076098, Training Accuracy= 0.98438
Iter 975360, Minibatch Loss= 0.099867, Training Accuracy= 0.95312
Iter 976640, Minibatch Loss= 0.059518, Training Accuracy= 0.96094
Iter 977920, Minibatch Loss= 0.067576, Training Accuracy= 0.99038
Iter 979200, Minibatch Loss= 0.075403, Training Accuracy= 0.98438
Iter 980480, Minibatch Loss= 0.099406, Training Accuracy= 0.95312
Iter 98176